In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 1. Xử lý và lưu dữ liệu VNM
# Đọc dữ liệu VNM
data_day_vnm = pd.read_csv('VNM.csv')
data_day_vnm['Ngày'] = pd.to_datetime(data_day_vnm['Ngày'], format='%d/%m/%Y')
data_day_vnm = data_day_vnm[(data_day_vnm['Ngày'] >= pd.to_datetime('2020-04-10')) & 
                            (data_day_vnm['Ngày'] <= pd.to_datetime('2025-04-10'))]
data_day_vnm['Lần cuối'] = data_day_vnm['Lần cuối'].astype(str).str.replace(',', '').astype(float)

# Tạo dữ liệu đầu tháng cho VNM
data_month_vnm = data_day_vnm.groupby(pd.Grouper(key='Ngày', freq='MS'))['Lần cuối'].first().reset_index()
data_month_vnm.columns = ['Date', 'VNM_Price']

# Đọc dữ liệu S&P 500 và VNINDEX
sp500_data = pd.read_csv('SP500_Full_Predictions_2015_2028.csv')
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])
vnindex_data = pd.read_csv('VNIINDEX_Full_Predictions_2015_2028.csv')
vnindex_data['Date'] = pd.to_datetime(vnindex_data['Date'])

# Tạo dữ liệu đầu tháng cho S&P 500 và VNINDEX
sp500_month = sp500_data.groupby(pd.Grouper(key='Date', freq='MS'))['Price'].first().reset_index()
vnindex_month = vnindex_data.groupby(pd.Grouper(key='Date', freq='MS'))['Lần cuối'].first().reset_index()

# Ghép dữ liệu tháng
data_month_vnm = data_month_vnm.merge(sp500_month, on='Date', how='left')
data_month_vnm = data_month_vnm.merge(vnindex_month, on='Date', how='left')
data_month_vnm = data_month_vnm.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_month_vnm = data_month_vnm.dropna()

# Ghép dữ liệu ngày
data_day_vnm = data_day_vnm.rename(columns={'Ngày': 'Date', 'Lần cuối': 'VNM_Price'})
data_day_vnm = data_day_vnm.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
data_day_vnm = data_day_vnm.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
data_day_vnm = data_day_vnm.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_day_vnm = data_day_vnm.dropna()

# Lưu dữ liệu đã xử lý
data_month_vnm.to_csv('VNM_Monthly_Processed.csv', index=False)
data_day_vnm.to_csv('VNM_Daily_Processed.csv', index=False)
print("Dữ liệu đã được xử lý và lưu vào 'VNM_Monthly_Processed.csv' và 'VNM_Daily_Processed.csv'")

# 2. Huấn luyện mô hình Prophet để dự đoán giá đầu tháng của VNM
model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.add_regressor('SP500_Price')
model_prophet.add_regressor('VNINDEX_Price')
model_prophet.fit(data_month_vnm.rename(columns={'Date': 'ds', 'VNM_Price': 'y'}))

# Tạo khung thời gian tương lai (từ 1/5/2025 đến 1/5/2028)
future_dates_month = pd.date_range(start='2025-05-01', end='2028-05-01', freq='MS')
future_df_month = pd.DataFrame({'ds': future_dates_month})

# Ghép giá dự đoán đầu tháng của S&P 500 và VNINDEX
future_df_month = future_df_month.merge(sp500_month[sp500_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.merge(vnindex_month[vnindex_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
future_df_month = future_df_month[['ds', 'SP500_Price', 'VNINDEX_Price']].ffill()

# Dự đoán giá đầu tháng của VNM
forecast = model_prophet.predict(future_df_month)
predicted_monthly_vnm = forecast[['ds', 'yhat']]
predicted_monthly_vnm.columns = ['Date', 'Predicted_Price']

# 3. Huấn luyện mô hình LSTM để dự đoán giá ngày
scaler = MinMaxScaler()
scaled_data_day = scaler.fit_transform(data_day_vnm[['VNM_Price', 'SP500_Price', 'VNINDEX_Price']])
if np.any(np.isnan(scaled_data_day)) or np.any(np.isinf(scaled_data_day)):
    scaled_data_day = np.nan_to_num(scaled_data_day, nan=0.0, posinf=0.0, neginf=0.0)

time_step = 300
def create_lstm_data(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_lstm_data(scaled_data_day, time_step)

X_train_tensor = torch.FloatTensor(X)
y_train_tensor = torch.FloatTensor(y).reshape(-1, 1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=50, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model_lstm = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lstm = model_lstm.to(device)

num_epochs = 30
for epoch in range(num_epochs):
    model_lstm.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model_lstm(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1.0)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# 4. Dự đoán giá ngày từ 10/4/2025 đến 1/5/2028 với scale liên tục
historical_data = scaled_data_day.tolist()
future_dates_daily = pd.date_range(start='2025-04-10', end='2028-05-01', freq='D')
daily_df = pd.DataFrame({'Date': future_dates_daily})

# Ghép giá dự đoán S&P 500 và VNINDEX cho ngày tương lai
daily_df = daily_df.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
daily_df = daily_df.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
daily_df = daily_df.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
daily_df = daily_df[['Date', 'SP500_Price', 'VNINDEX_Price']].ffill()

# Chuẩn hóa dữ liệu dự đoán (thêm cột placeholder cho VNM_Price)
daily_df['VNM_Price'] = 0  # Placeholder sẽ được cập nhật sau
scaled_future_data = scaler.transform(daily_df[['VNM_Price', 'SP500_Price', 'VNINDEX_Price']])

# Kiểm tra và bù/cắt dữ liệu để khớp với future_dates_daily
if len(scaled_future_data) > len(future_dates_daily):
    scaled_future_data = scaled_future_data[:len(future_dates_daily)]
elif len(scaled_future_data) < len(future_dates_daily):
    # Bù thêm bằng giá trị cuối cùng nếu thiếu
    last_row = scaled_future_data[-1]
    additional_rows = np.tile(last_row, (len(future_dates_daily) - len(scaled_future_data), 1))
    scaled_future_data = np.vstack((scaled_future_data, additional_rows))

model_lstm.eval()
predictions_daily = []
current_window = historical_data[-time_step:]

# Lấy giá thực tế tại 10/4/2025
last_actual_price = data_day_vnm[data_day_vnm['Date'] == pd.to_datetime('2025-04-10')]['VNM_Price'].iloc[0]

# Dự đoán từng ngày
total_days = len(future_dates_daily)
for i in range(total_days):
    current_window_tensor = torch.FloatTensor(current_window[-time_step:]).unsqueeze(0).to(device)
    with torch.no_grad():
        pred = model_lstm(current_window_tensor).cpu().numpy()[0, 0]
        predictions_daily.append(pred)
        # Cập nhật cửa sổ với giá dự đoán
        new_window_data = np.append(current_window_tensor.cpu().numpy()[0, 1:, :], 
                                   [[pred, scaled_future_data[i, 1], scaled_future_data[i, 2]]], axis=0)
        current_window.append([pred, scaled_future_data[i, 1], scaled_future_data[i, 2]])

# Đảo ngược chuẩn hóa toàn bộ dự đoán
predictions_daily = np.array(predictions_daily).reshape(-1, 1)
predictions_rescaled = scaler.inverse_transform(
    np.hstack((predictions_daily, scaled_future_data[:len(predictions_daily), 1:]))
)[:, 0]

# Scale giá ngày để khớp với giá đầu tháng
adjusted_predictions = predictions_rescaled.copy()
current_segment_start_idx = 0
for j in range(len(predicted_monthly_vnm)):
    if j == 0:
        start_date = pd.to_datetime('2025-04-10')
        end_date = pd.to_datetime('2025-04-30')
        next_month_start = predicted_monthly_vnm['Date'].iloc[j]
    else:
        start_date = predicted_monthly_vnm['Date'].iloc[j - 1]
        end_date = predicted_monthly_vnm['Date'].iloc[j] - pd.Timedelta(days=1)
        next_month_start = predicted_monthly_vnm['Date'].iloc[j]

    if j == len(predicted_monthly_vnm) - 1:
        end_date = pd.to_datetime('2028-05-01')

    if start_date > end_date:
        continue

    segment_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    segment_indices = [i for i, date in enumerate(future_dates_daily) if date in segment_dates]
    if not segment_indices:
        continue

    segment_start_idx = segment_indices[0]
    segment_end_idx = segment_indices[-1]
    segment_predictions = adjusted_predictions[segment_start_idx:segment_end_idx + 1]

    if j == 0:  # Đoạn đầu tiên từ 10/4/2025 đến 30/4/2025
        monthly_price_next = predicted_monthly_vnm[predicted_monthly_vnm['Date'] == next_month_start]['Predicted_Price'].iloc[0]
        daily_price_at_start = last_actual_price  # Giá thực tế tại 10/4/2025
        daily_price_at_end = segment_predictions[-1]

        adjustment_at_start = 0  # Không điều chỉnh giá tại 10/4/2025
        adjustment_at_end = monthly_price_next - daily_price_at_end
        segment_length = len(segment_predictions)
        adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
        adjusted_predictions[segment_start_idx:segment_end_idx + 1] = segment_predictions + adjustments
    else:  # Các đoạn sau từ 1/5/2025 trở đi
        monthly_price_current = predicted_monthly_vnm[predicted_monthly_vnm['Date'] == start_date]['Predicted_Price'].iloc[0]
        if end_date >= next_month_start - pd.Timedelta(days=1):
            monthly_price_next = predicted_monthly_vnm[predicted_monthly_vnm['Date'] == next_month_start]['Predicted_Price'].iloc[0]
            daily_price_at_end = segment_predictions[-1]
            daily_price_at_start = segment_predictions[0]

            adjustment_at_start = monthly_price_current - daily_price_at_start
            adjustment_at_end = monthly_price_next - daily_price_at_end
            segment_length = len(segment_predictions)
            adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
            adjusted_predictions[segment_start_idx:segment_end_idx + 1] = segment_predictions + adjustments
        else:
            adjustment_at_start = monthly_price_current - segment_predictions[0]
            adjusted_predictions[segment_start_idx:segment_end_idx + 1] = segment_predictions + adjustment_at_start

    current_segment_start_idx = segment_end_idx + 1

# 5. Tạo DataFrame kết hợp dữ liệu lịch sử và dự đoán
combined_data = pd.concat([data_day_vnm[data_day_vnm['Date'] <= pd.to_datetime('2025-04-10')], 
                          pd.DataFrame({'Date': future_dates_daily, 'VNM_Price': adjusted_predictions})])
combined_data = combined_data[(combined_data['Date'] >= pd.to_datetime('2020-04-10')) & 
                              (combined_data['Date'] <= pd.to_datetime('2028-05-01'))]
combined_data = combined_data.reset_index(drop=True)

# Lưu kết quả vào file CSV
combined_data.to_csv('VNM_Full_Predictions_2020_2028.csv', index=False)
print("Dữ liệu đầy đủ từ 10/4/2020 đến 1/5/2028 đã được lưu vào 'VNM_Full_Predictions_2020_2028.csv'")

# Biểu đồ tổng quan từ 10/4/2020 đến 1/5/2028
plt.figure(figsize=(14, 7))
plt.plot(data_month_vnm['Date'], data_month_vnm['VNM_Price'], label='Dữ liệu lịch sử (Tháng)', color='blue')
plt.plot(data_day_vnm['Date'], data_day_vnm['VNM_Price'], label='Dữ liệu lịch sử (Ngày)', color='lightblue', alpha=0.5)

predicted_monthly_vnm_filtered = predicted_monthly_vnm[predicted_monthly_vnm['Date'] >= pd.to_datetime('2025-04-10')]
plt.plot(predicted_monthly_vnm_filtered['Date'], predicted_monthly_vnm_filtered['Predicted_Price'], 
         label='Dự đoán (Tháng)', color='orange', marker='o')

plt.plot(combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['Date'], 
         combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['VNM_Price'], 
         label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán VNM: Tháng và Ngày (Scale liên tục)')
plt.xlabel('Ngày')
plt.ylabel('Giá VNM')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('VNM_Prediction_Chart_Full.png')
plt.close()

# Biểu đồ chi tiết từ 10/4/2025 đến 1/5/2028
mask_detail = (combined_data['Date'] >= pd.to_datetime('2025-04-10')) & (combined_data['Date'] <= pd.to_datetime('2028-05-01'))
combined_data_detail = combined_data[mask_detail]
month_df_detail = predicted_monthly_vnm[
    (predicted_monthly_vnm['Date'] >= pd.to_datetime('2025-04-10')) & 
    (predicted_monthly_vnm['Date'] <= pd.to_datetime('2028-05-01'))
]

plt.figure(figsize=(14, 7))
plt.plot(month_df_detail['Date'], month_df_detail['Predicted_Price'], label='Dự đoán (Tháng)', color='orange', marker='o')
plt.plot(combined_data_detail['Date'], combined_data_detail['VNM_Price'], label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán VNM: Tháng và Ngày (10/4/2025 - 1/5/2028)')
plt.xlabel('Ngày')
plt.ylabel('Giá VNM')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('VNM_Prediction_Chart_Detail_2025_2028.png')
plt.close()

print("Biểu đồ tổng quan đã được lưu vào 'VNM_Prediction_Chart_Full.png'")
print("Biểu đồ chi tiết (10/4/2025 - 1/5/2028) đã được lưu vào 'VNM_Prediction_Chart_Detail_2025_2028.png'")

Importing plotly failed. Interactive plots will not work.


Dữ liệu đã được xử lý và lưu vào 'VNM_Monthly_Processed.csv' và 'VNM_Daily_Processed.csv'


23:53:23 - cmdstanpy - INFO - Chain [1] start processing
23:53:23 - cmdstanpy - INFO - Chain [1] done processing


Epoch [1/30], Loss: 0.064919
Epoch [2/30], Loss: 0.013187
Epoch [3/30], Loss: 0.004519
Epoch [4/30], Loss: 0.003025
Epoch [5/30], Loss: 0.003468
Epoch [6/30], Loss: 0.001707
Epoch [7/30], Loss: 0.001772
Epoch [8/30], Loss: 0.001475
Epoch [9/30], Loss: 0.001920
Epoch [10/30], Loss: 0.002447
Epoch [11/30], Loss: 0.003184
Epoch [12/30], Loss: 0.002060
Epoch [13/30], Loss: 0.001743
Epoch [14/30], Loss: 0.002223
Epoch [15/30], Loss: 0.001820
Epoch [16/30], Loss: 0.001183
Epoch [17/30], Loss: 0.002888
Epoch [18/30], Loss: 0.003306
Epoch [19/30], Loss: 0.001369
Epoch [20/30], Loss: 0.001692
Epoch [21/30], Loss: 0.001957
Epoch [22/30], Loss: 0.001493
Epoch [23/30], Loss: 0.001873
Epoch [24/30], Loss: 0.001583
Epoch [25/30], Loss: 0.001471
Epoch [26/30], Loss: 0.001795
Epoch [27/30], Loss: 0.001748
Epoch [28/30], Loss: 0.001213
Epoch [29/30], Loss: 0.001308
Epoch [30/30], Loss: 0.001152
Dữ liệu đầy đủ từ 10/4/2020 đến 1/5/2028 đã được lưu vào 'VNM_Full_Predictions_2020_2028.csv'
Biểu đồ tổng qu